# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [55]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [40]:
df = pd.read_csv('previsao_de_renda.csv')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [42]:
# Definindo as variáveis preditoras (X) e alvo (y)
X = df.drop(columns=['Unnamed: 0', 'id_cliente', 'renda', 'data_ref'])
y = df['renda']

# Separando em treino e teste (25% para teste, 75% para treinamento)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

(X_train.shape, X_test.shape)

((11250, 11), (3750, 11))

In [43]:
# Convertendo variáveis categóricas em variáveis dummies (one-hot encoding)
X_encoded = pd.get_dummies(X, drop_first=True)

# Verificando se a codificação funcionou corretamente
X_encoded.head()

,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,sexo_M,tipo_renda_Bolsista,tipo_renda_Empresário,tipo_renda_Pensionista,...,educacao_Superior incompleto,estado_civil_Separado,estado_civil_Solteiro,estado_civil_União,estado_civil_Viúvo,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,tipo_residencia_Governamental
0,False,True,0,26,6.602740,1.0,False,False,True,False,...,False,False,True,False,False,True,False,False,False,False
1,True,True,0,28,7.183562,2.0,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,True,True,0,35,0.838356,2.0,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
3,False,True,1,30,4.846575,3.0,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,True,False,0,33,4.293151,1.0,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [44]:
alpha_values = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

In [46]:
# Criando o imputer para substituir valores ausentes (NaN) por médias das colunas
imputer = SimpleImputer(strategy='mean')

# Aplicando o imputer aos conjuntos de treino e teste
X_train_imputed = imputer.fit_transform(X_train_encoded)
X_test_imputed = imputer.transform(X_test_encoded)

# Recalculando os scores com as variáveis imutadas
ridge_scores = {}

for alpha in alpha_values:
    model = Ridge(alpha=alpha)
    model.fit(X_train_imputed, y_train)
    
    y_pred = model.predict(X_test_imputed)
    
    r2 = r2_score(y_test, y_pred)
    
    ridge_scores[alpha] = r2

ridge_scores

{0: 0.2691159056847583,
 0.001: 0.2691159145559312,
 0.005: 0.2691159500045429,
 0.01: 0.26911599423430144,
 0.05: 0.2691163448655334,
 0.1: 0.26911677529614875}

In [54]:
# Inicializando um dicionário para armazenar os scores
lasso_scores = {}

# Loop sobre os diferentes valores de alpha
for alpha in alpha_values:
    # Criando e ajustando o modelo Lasso
    model = Lasso(alpha=alpha)
    model.fit(X_train_imputed, y_train)
    
    # Predizendo no conjunto de teste
    y_pred = model.predict(X_test_imputed)
    
    # Calculando o R-quadrado
    r2 = r2_score(y_test, y_pred)
    
    # Armazenando os resultados
    lasso_scores[alpha] = r2

lasso_scores

C:\Users\Matheus Cerqueira\AppData\Local\Temp\ipykernel_23328\4123808859.py:8: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  model.fit(X_train_imputed, y_train)
E:\Users\Matheus Cerqueira\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
E:\Users\Matheus Cerqueira\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.858e+11, tolerance: 7.723e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  mo

{0: 0.2691159441765779,
 0.001: 0.2691160348452796,
 0.005: 0.2691163940799389,
 0.01: 0.2691168353829293,
 0.05: 0.26912005619351154,
 0.1: 0.2691233081733694}

In [52]:
# Criando o modelo de regressão linear
linear_model = LinearRegression()

# Configurando o SequentialFeatureSelector para seleção forward (forward stepwise selection)
sfs = SequentialFeatureSelector(linear_model, n_features_to_select='auto', direction='forward', scoring='r2', cv=5)

# Ajustando o modelo com seleção de variáveis no conjunto de treino
sfs.fit(X_train_imputed, y_train)

# Verificando as variáveis selecionadas
selected_features = sfs.get_support()

# Exibindo as variáveis selecionadas
selected_column_names = X_encoded.columns[selected_features]
selected_column_names


Index(['posse_de_imovel', 'idade', 'tempo_emprego', 'qt_pessoas_residencia',
       'sexo_M', 'tipo_renda_Bolsista', 'tipo_renda_Empresário',
       'tipo_renda_Pensionista', 'educacao_Superior completo',
       'educacao_Superior incompleto', 'estado_civil_Solteiro',
       'estado_civil_União'],
      dtype='object')

In [53]:
# Aplicando o modelo com as variáveis selecionadas
X_train_selected = X_train_imputed[:, selected_features]
X_test_selected = X_test_imputed[:, selected_features]

# Ajustando o modelo de regressão linear nas variáveis selecionadas
linear_model.fit(X_train_selected, y_train)

# Predizendo no conjunto de teste com as variáveis selecionadas
y_pred_selected = linear_model.predict(X_test_selected)

# Calculando o R-quadrado para o modelo com as variáveis selecionadas
r2_selected = r2_score(y_test, y_pred_selected)
r2_selected


0.2697471820649183

In [57]:
# Criando e ajustando um modelo de árvore de decisão
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train_imputed, y_train)

# Fazendo previsões no conjunto de teste
y_pred_tree = tree_model.predict(X_test_imputed)

# Calculando o R-quadrado para a árvore de decisão
r2_tree = r2_score(y_test, y_pred_tree)
r2_tree


0.26042899259730756